In [1]:
# Import the dependencies.

import gmaps.datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# Files to load
vacation_data_to_load = "weather_data/WeatherPy_vacation.csv"

In [3]:
# Read the school data file and store it in a Pandas DataFrame.
vacation_data_df = pd.read_csv(vacation_data_to_load)
vacation_data_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,1,Padang,ID,65.68,-0.92,100.36,Mercure Padang
1,2,Chuy,UY,65.05,-33.69,-53.46,Nuevo Hotel Plaza
2,5,Vallenar,CL,68.27,-28.58,-70.76,Humacao Bed & Breakfast
3,6,Jalu,LY,70.48,29.03,21.55,Jalu Hotel
4,7,Vaini,IN,67.01,15.34,74.49,Dandeli Lake County


In [4]:
vacation_data_df_filter= vacation_data_df.loc[vacation_data_df['City'].isin(["Pacific Grove","Half Moon Bay","Lompoc","Pacifica"])]
vacation_data_df_filter

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
42,69,Lompoc,US,80.01,34.64,-120.46,Holiday Inn Express Lompoc
70,118,Pacific Grove,US,75.99,36.62,-121.92,Lovers Point Inn
224,424,Pacifica,US,75.00,37.61,-122.49,Lighthouse Hotel
260,484,Half Moon Bay,US,75.00,37.46,-122.43,"The Ritz-Carlton, Half Moon Bay"


In [5]:
# Latitude-longitude pairs
Pacific_Grove = (36.62, -121.92)
Half_Moon_Bay = (37.46, -122.43)
Pacifica = (37.61, -122.49)
Lompoc=(34.64,-120.46)

fig = gmaps.figure()
Lompoc2Pacifica = gmaps.directions_layer(Lompoc,Pacifica)
fig.add_layer(Lompoc2Pacifica)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
fig = gmaps.figure()
Lompoc2Pacifica_via_Pacific_Grove_Half_Moon_Bay = gmaps.directions_layer(
        Lompoc, Pacifica, waypoints=[Pacific_Grove , Half_Moon_Bay],
        travel_mode='DRIVING')
fig.add_layer(Lompoc2Pacifica_via_Pacific_Grove_Half_Moon_Bay)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
vacation_data_df_filter

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
42,69,Lompoc,US,80.01,34.64,-120.46,Holiday Inn Express Lompoc
70,118,Pacific Grove,US,75.99,36.62,-121.92,Lovers Point Inn
224,424,Pacifica,US,75.00,37.61,-122.49,Lighthouse Hotel
260,484,Half Moon Bay,US,75.00,37.46,-122.43,"The Ritz-Carlton, Half Moon Bay"


In [8]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()


In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_data_df_filter[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
42,Lompoc,US,80.01,34.64,-120.46,
70,Pacific Grove,US,75.99,36.62,-121.92,
224,Pacifica,US,75.00,37.61,-122.49,
260,Half Moon Bay,US,75.00,37.46,-122.43,


In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to
      #the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [11]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [12]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
42,Lompoc,US,80.01,34.64,-120.46,Holiday Inn Express Lompoc
70,Pacific Grove,US,75.99,36.62,-121.92,Lovers Point Inn
224,Pacifica,US,75.00,37.61,-122.49,Lighthouse Hotel
260,Half Moon Bay,US,75.00,37.46,-122.43,"The Ritz-Carlton, Half Moon Bay"


In [51]:

locations = hotel_df [["Lat", "Lng"]]
cities = hotel_df["City"]
fig = gmaps.figure(center=(36.5, -121.0), zoom_level=6.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, /
#             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
plt.savefig("WeatherPy_travel_map_markers.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>